In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfd

### Loading the data 

In [2]:
mnist_dataset, mnist_info =tfd.load(name='mnist',with_info=True, as_supervised=True)

### Splitting the test, train and validation data (already labelled) 

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [4]:
mnist_train

<DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [5]:
num_validation=0.1*mnist_info.splits['train'].num_examples
num_validation=tf.cast(num_validation,tf.int64)

In [6]:
num_test=mnist_info.splits['test'].num_examples
num_test=tf.cast(num_test,tf.int64)

In [7]:
num_validation

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

### Scaling the data 

In [8]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

In [9]:
scaledtrainandvalidation=mnist_train.map(scale)
scaledtest=mnist_test.map(scale)

In [10]:
scaledtest

<DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

### Shuffling the data to avoid repitition 

In [11]:
BUFFER_SIZE=10000
shuffle_scaledtrainandvalidation=scaledtrainandvalidation.shuffle(BUFFER_SIZE)
shuffle_scaledtest=scaledtest.shuffle(BUFFER_SIZE)

In [12]:
validation_data=shuffle_scaledtrainandvalidation.take(num_validation)

In [13]:
train_data=shuffle_scaledtrainandvalidation.skip(num_validation)

In [14]:
BATCH_SIZE=100

In [15]:
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation)
test_data=shuffle_scaledtest.batch(num_test)
validation_inputs,validation_targets=next(iter(validation_data))

### Building model with two layers and 100 neurons on each 

In [23]:
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

### Chossing the optimizer 

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Fitting the model 

In [25]:
model.fit(train_data,epochs=5, validation_data=(validation_inputs,validation_targets),validation_steps=10, verbose=2)

Epoch 1/5
540/540 - 13s - loss: 0.3299 - accuracy: 0.9058 - val_loss: 0.1737 - val_accuracy: 0.9507
Epoch 2/5
540/540 - 13s - loss: 0.1435 - accuracy: 0.9580 - val_loss: 0.1236 - val_accuracy: 0.9630
Epoch 3/5
540/540 - 13s - loss: 0.1020 - accuracy: 0.9694 - val_loss: 0.0917 - val_accuracy: 0.9730
Epoch 4/5
540/540 - 13s - loss: 0.0769 - accuracy: 0.9767 - val_loss: 0.0774 - val_accuracy: 0.9787
Epoch 5/5
540/540 - 15s - loss: 0.0641 - accuracy: 0.9797 - val_loss: 0.0646 - val_accuracy: 0.9800


### Validating the test data for final accuracy

In [26]:
model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.0826 - accuracy: 0.9738

[0.08256971091032028, 0.9738]